# Speech Agent

In [4]:
import os
import logging
import textwrap
from typing import List
from pydantic import BaseModel, Field


from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool, WebsiteSearchTool, ScrapeWebsiteTool


from IPython.display import display, Markdown


def load_env():
    
    pass

# Set up logging configuration
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# ================================
# Agent Configurations
# ================================
agents_config = {
    "company_info_agent": {
        "role": "Company Information Specialist",
        "goal": "Extract comprehensive background, industry details, key achievements, and overall impact for a given company.",
        "backstory": (
            "As a specialist in company analysis, you gather and summarize detailed information about companies to provide "
            "insights that will help craft an engaging speech."
        ),
        "allow_delegation": False,
        "verbose": True,
    },
    "country_info_agent": {
        "role": "Country Information Specialist",
        "goal": "Gather key data on the economy, culture, and business environment of a given country.",
        "backstory": (
            "As an expert in country profiling, you analyze essential national metrics and cultural aspects to provide valuable "
            "context for a well-rounded speech."
        ),
        "allow_delegation": False,
        "verbose": True,
    },
    "speech_generator_agent": {
        "role": "Speech Generator",
        "goal": (
            "Create a well-structured, engaging, and informative speech for an invited guest by integrating company and country data."
        ),
        "backstory": (
            "As an experienced speechwriter, you craft compelling speeches that seamlessly combine detailed company information "
            "with insights about the country to produce a coherent and engaging narrative."
        ),
        "allow_delegation": False,
        "verbose": True,
    },
    "quality_assurance_agent": {
        "role": "Content Quality Assurer",
        "goal": (
            "Review and refine the generated speech to ensure clarity, factual accuracy, and overall engagement, while ensuring "
            "the content is well-formatted in markdown."
        ),
        "backstory": (
            "As the quality assurance specialist, you meticulously proofread and optimize content to meet high standards of quality and "
            "presentation."
        ),
        "allow_delegation": False,
        "verbose": True,
    },
}

# ================================
# Task Configurations
# ================================
tasks_config = {
    "retrieve_company_info": {
        "description": (
            "Retrieve comprehensive company information including background, industry details, achievements, and overall impact for {company}."
        ),
        "expected_output": (
            "A detailed summary of {company} covering its background, industry position, notable achievements, and overall impact."
        ),
    },
    "retrieve_country_info": {
        "description": (
            "Gather key data on the economy, culture, and business environment for {country}."
        ),
        "expected_output": (
            "A detailed summary of {country} covering economic trends, cultural highlights, and the business environment."
        ),
    },
    "generate_speech": {
        "description": (
            "Based on the company and country information retrieved, generate a well-structured and engaging speech for an invited guest. "
            "Integrate insights and contextual data into a compelling narrative."
        ),
        "expected_output": (
            "A comprehensive speech that seamlessly combines the company and country information into a clear, engaging narrative suitable for an invited guest."
        ),
    },
    "quality_assurance": {
        "description": (
            "Review and refine the generated speech to ensure it meets the highest standards of clarity, engagement, and factual accuracy. "
            "Ensure the speech is well-formatted in markdown with appropriate headers and bullet points."
        ),
        "expected_output": (
            "A finalized version of the speech, thoroughly reviewed and formatted in markdown, that is clear, engaging, and informative."
        ),
    },
}

# ================================
# Pydantic Model for Output
# ================================
class SpeechOutput(BaseModel):
    speech: str = Field(..., description="The final generated speech in markdown format.")

# ================================
# Utility function to print the speech neatly
# ================================
def print_speech(speech_output: SpeechOutput) -> None:
    # Use model_dump() to avoid the deprecated dict() method.
    speech = speech_output.model_dump().get("speech", "")
    if not speech:
        logging.warning("No speech found in the output.")
        return
    wrapped_speech = textwrap.fill(speech, width=80)
    print("\nFinal Speech:")
    print("=" * 80)
    print(wrapped_speech)
    print("=" * 80)

# ================================
# Main Function
# ================================
def main():
    
    load_env()

    
    #os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'
    groq_llm = "groq/llama-3.3-70b-versatile"

    # ---------------------------
    # Create Agents with Tools
    # ---------------------------
    company_info_agent = Agent(
        config=agents_config['company_info_agent'],
        tools=[SerperDevTool(), ScrapeWebsiteTool(), WebsiteSearchTool()],
        llm=groq_llm,
    )

    country_info_agent = Agent(
        config=agents_config['country_info_agent'],
        tools=[SerperDevTool(), WebsiteSearchTool()],
        llm=groq_llm,
    )

    speech_generator_agent = Agent(
        config=agents_config['speech_generator_agent'],
        tools=[SerperDevTool(), WebsiteSearchTool()],
        llm=groq_llm,
    )

    quality_assurance_agent = Agent(
        config=agents_config['quality_assurance_agent'],
    )

    # ---------------------------
    # Create Tasks
    # ---------------------------
    retrieve_company_info_task = Task(
        config=tasks_config['retrieve_company_info'],
        agent=company_info_agent
    )

    retrieve_country_info_task = Task(
        config=tasks_config['retrieve_country_info'],
        agent=country_info_agent
    )

    generate_speech_task = Task(
        config=tasks_config['generate_speech'],
        agent=speech_generator_agent,
        context=[retrieve_company_info_task, retrieve_country_info_task]
    )

    quality_assurance_task = Task(
        config=tasks_config['quality_assurance'],
        agent=quality_assurance_agent,
        output_pydantic=SpeechOutput
    )

    # ---------------------------
    # Assemble Crew and Execute Tasks
    # ---------------------------
    speech_creation_crew = Crew(
        agents=[
            company_info_agent,
            country_info_agent,
            speech_generator_agent,
            quality_assurance_agent,
        ],
        tasks=[
            retrieve_company_info_task,
            retrieve_country_info_task,
            generate_speech_task,
            quality_assurance_task
        ],
        verbose=True
    )

    # Define input details for the speech 
    input_subject = {
        'company': 'Ethio Telecom',
        'country': 'Ethiopia'
    }
    logging.info("Kicking off the speech creation crew.")
    result = speech_creation_crew.kickoff(inputs=input_subject)
    logging.info("Crew execution completed.")

    # Debug: Print raw result 
    print(result)

    # ---------------------------
    # Display the Final Speech
    # ---------------------------
    if hasattr(result, 'pydantic'):
        speech_output: SpeechOutput = result.pydantic
        
        display(Markdown(speech_output.speech))
        
        print_speech(speech_output)
    else:
        logging.error("The result does not contain a valid 'pydantic' attribute.")

# Execute the main function to run the workflow
main()


2025-03-05 15:19:58,720 - 140230655523712 - sqlite.py-sqlite:111 - WARNING: ⚠️ It looks like you upgraded from a version below 0.6 and could benefit from vacuuming your database. Run chromadb utils vacuum --help for more information.


# Agent: Company Information Specialist
## Task: Retrieve comprehensive company information including background, industry details, achievements, and overall impact for Ethio Telecom.


# Agent: Company Information Specialist
## Thought: Thought: To gather comprehensive information about Ethio Telecom, I need to start by searching the internet for a general overview of the company, including its background, industry position, and notable achievements.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"Ethio Telecom company overview\"}"
## Tool Output: 

Search results: Title: Profile – ethiotelecom
Link: https://www.ethiotelecom.et/profile/
Snippet: We are Africa's Pioneer telecom provider, offering reliable connectivity, advanced technology, and financial services to simplify life as we drive ...
---
Title: Ethio telecom - Wikipedia
Link: https://en.wikipedia.org/wiki/Ethio_telecom
Snippet: Ethio telecom is owned by the Ethiopian government and maintains a monopo

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  7.01it/s]




# Agent: Company Information Specialist
## Thought: I decline
Settings
 Ethio Telecom - Africa’s Pioneer in Connectivity 
 Who We are 
History
 Annual Financial Statement 
 Profile 
CSR 
We Are Africa’s Pioneer Telecom Provider, Offering Reliable Connectivity, Advanced Technology, And Financial Services To Simplify Life As We Drive Ethiopia’s Digital Transformation And Empower Communities With Innovative Solutions. 
 Skip to content 
+251-994/+251-980
994@ethionet.et 
8994 
Addis Ababa, Ethiopia
My Ethiotel
My Ethiotel
 Search
Close 
Eng 
አማ 
Oro 
teleshares 
telebirr 
Cloud solutions 
telecloud 
Partnership cloud solutions 
zergaw cloud solution 
Website builder solution 
Online Services 
National ID 
Developer Portal(telebirr) 
teleHub 
telegebeya 
Postpaid bill complaint 
ecare 
DNS and webhosting 
teleforum 
Interactive self-support 
5G coverage map 
Personal 
Getting Started 
teledrive 
5G Mobile Packages 
Social Media packages 
Long Validity Package 
Mobile Packages 
Fixed BB I



# Agent: Country Information Specialist
## Thought: Thought: To gather key data on the economy, culture, and business environment of Ethiopia, I should start by searching for general information about the country's economy.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"Ethiopia economy overview\"}"
## Tool Output: 

Search results: Title: Ethiopia Overview: Development news, research, data | World Bank
Link: https://www.worldbank.org/en/country/ethiopia/overview
Snippet: Ethiopia's strong growth record has been reflected in average real growth of nearly 10% per year between 2004-18, one of the highest rates in ...
---
Title: Ethiopia Economic Outlook | African Development Bank Group
Link: https://www.afdb.org/en/countries/east-africa/ethiopia/ethiopia-economic-outlook
Snippet: Ethiopia's economy grew 7.1% in 2022/23, up from 6.4% in 2021/22, led by 7.9% growth in the services sector, which accounts for 40% of GDP.
---
Title: Economy of Ethiopia - Wikipedia




# Agent: Country Information Specialist
## Final Answer: 
Ethiopia is a country with a rich cultural heritage and a growing economy. The country has a unique cultural heritage, with over 80 different ethnic groups, each with their own language and traditions. The Ethiopian Orthodox Church plays a significant role in the country's culture and traditions. The economy of Ethiopia is primarily based on agriculture, with the country being one of the largest producers of coffee, cereals, and other crops. The country has also made significant progress in recent years in terms of industrialization and modernization, with a growing manufacturing sector and an increasing number of foreign investments. The business environment in Ethiopia is relatively stable, with a growing number of foreign investors and a government that is committed to promoting economic development. However, there are still challenges to doing business in Ethiopia, including a lack of infrastructure, corruption, and bureau



# Agent: Content Quality Assurer
## Final Answer: 
# Speech on the Journey of Ethio Telecom

Ladies and gentlemen, esteemed guests, and fellow citizens of Ethiopia, 

It is my honor to stand before you today to discuss the remarkable journey of **Ethio Telecom**, the premier telecom provider in our great nation. As we gather here today, I would like to take a moment to acknowledge the significant contributions that Ethio Telecom has made to the growth and development of our country.

## A Brief History

Established in **2010**, with roots dating back to **1894**, Ethio Telecom has undergone significant transformations, expanding its services to include:

- **Reliable connectivity**
- **Advanced technology**
- **Financial services**

The company's commitment to providing exceptional services has empowered communities, driven economic growth, and simplified life for millions of Ethiopians.

## Looking to the Future

As we look to the future, we are excited about the prospects of contin

Ladies and gentlemen, esteemed guests, and fellow citizens of Ethiopia, 

It is my honor to stand before you today to discuss the remarkable journey of **Ethio Telecom**, the premier telecom provider in our great nation. As we gather here today, I would like to take a moment to acknowledge the significant contributions that Ethio Telecom has made to the growth and development of our country.

## A Brief History

Established in **2010**, with roots dating back to **1894**, Ethio Telecom has undergone significant transformations, expanding its services to include:

- **Reliable connectivity**
- **Advanced technology**
- **Financial services**

The company's commitment to providing exceptional services has empowered communities, driven economic growth, and simplified life for millions of Ethiopians.

## Looking to the Future

As we look to the future, we are excited about the prospects of continued growth and development. The Ethiopian government has implemented policies aimed at promoting economic development, and the business environment in Ethiopia is relatively stable, with a growing number of foreign investors. However, we also acknowledge the challenges that still exist, including:

- **Lack of infrastructure**
- **Corruption**
- **Bureaucratic hurdles**

## Ethio Telecom's Commitment

Despite these challenges, Ethio Telecom remains committed to its mission of providing innovative solutions to drive digital transformation and empower communities. Some of the company's remarkable efforts include:

- **Expansion of its 5G network**
- **Introduction of cloud-based solutions**
- **Launch of digital financial services**

These initiatives exemplify Ethio Telecom’s dedication to staying ahead of the curve.

## Economic and Social Impact

As the largest telecom provider in Ethiopia, Ethio Telecom plays a critical role in the country's digital transformation. With a significant market share and a wide range of services—including mobile voice, data and internet, fixed broadband, and fixed voice services—the company has made substantial contributions to the economy.

In addition to its economic contributions, Ethio Telecom has positively impacted the social fabric of our society. The company's commitment to **corporate social responsibility** is evident in its focus on:

- **Education**
- **Health**
- **Environmental protection**
- **Humanitarian services**

## Conclusion and Call to Action

As we move forward, we are excited about the prospect of Ethio Telecom's continued growth and development. The company's plans to expand its services, invest in new technologies, and promote digital inclusion will undoubtedly have a positive effect on the lives of millions of Ethiopians.

In conclusion, I would like to express my gratitude to Ethio Telecom for its tireless efforts to drive digital transformation and empower communities in Ethiopia. As we celebrate the company's achievements, we also recognize the significant challenges that still exist and the need for continued investment and innovation.

Let us work together to promote economic development, improve the business environment, and address the challenges that persist. With the support of companies like Ethio Telecom, I have no doubt that Ethiopia will continue to grow and prosper, becoming an increasingly attractive destination for foreign investors and tourists.

Thank you.


Final Speech:
Ladies and gentlemen, esteemed guests, and fellow citizens of Ethiopia,   It is
my honor to stand before you today to discuss the remarkable journey of **Ethio
Telecom**, the premier telecom provider in our great nation. As we gather here
today, I would like to take a moment to acknowledge the significant
contributions that Ethio Telecom has made to the growth and development of our
country.  ## A Brief History  Established in **2010**, with roots dating back to
**1894**, Ethio Telecom has undergone significant transformations, expanding its
services to include:  - **Reliable connectivity** - **Advanced technology** -
**Financial services**  The company's commitment to providing exceptional
services has empowered communities, driven economic growth, and simplified life
for millions of Ethiopians.  ## Looking to the Future  As we look to the future,
we are excited about the prospects of continued growth and development. The
Ethiopian government has implemented policies ai

In [6]:
import os
import logging
import textwrap
from typing import List
from pydantic import BaseModel, Field


from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool, WebsiteSearchTool, ScrapeWebsiteTool


from IPython.display import display, Markdown


def load_env():
    
    pass

# Set up logging configuration
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# ================================
# Agent Configurations
# ================================
agents_config = {
    "company_info_agent": {
        "role": "Company Information Specialist",
        "goal": "Extract comprehensive background, industry details, key achievements, and overall impact for a given company.",
        "backstory": (
            "As a specialist in company analysis, you gather and summarize detailed information about companies to provide "
            "insights that will help craft an engaging speech."
        ),
        "allow_delegation": False,
        "verbose": True,
    },
    "country_info_agent": {
        "role": "Country Information Specialist",
        "goal": "Gather key data on the economy, culture, and business environment of a given country.",
        "backstory": (
            "As an expert in country profiling, you analyze essential national metrics and cultural aspects to provide valuable "
            "context for a well-rounded speech."
        ),
        "allow_delegation": False,
        "verbose": True,
    },
    "speech_generator_agent": {
        "role": "Speech Generator",
        "goal": (
            "Create a well-structured, engaging, and informative speech for an invited guest by integrating company and country data."
        ),
        "backstory": (
            "As an experienced speechwriter, you craft compelling speeches that seamlessly combine detailed company information "
            "with insights about the country to produce a coherent and engaging narrative."
        ),
        "allow_delegation": False,
        "verbose": True,
    },
    "quality_assurance_agent": {
        "role": "Content Quality Assurer",
        "goal": (
            "Review and refine the generated speech to ensure clarity, factual accuracy, and overall engagement, while ensuring "
            "the content is well-formatted in markdown."
        ),
        "backstory": (
            "As the quality assurance specialist, you meticulously proofread and optimize content to meet high standards of quality and "
            "presentation."
        ),
        "allow_delegation": False,
        "verbose": True,
    },
}

# ================================
# Task Configurations
# ================================
tasks_config = {
    "retrieve_company_info": {
        "description": (
            "Retrieve comprehensive company information including background, industry details, achievements, and overall impact for {company}."
        ),
        "expected_output": (
            "A detailed summary of {company} covering its background, industry position, notable achievements, and overall impact."
        ),
    },
    "retrieve_country_info": {
        "description": (
            "Gather key data on the economy, culture, and business environment for {country}."
        ),
        "expected_output": (
            "A detailed summary of {country} covering economic trends, cultural highlights, and the business environment."
        ),
    },
    "generate_speech": {
        "description": (
            "Based on the company and country information retrieved, generate a well-structured and engaging speech for an invited guest. "
            "Integrate insights and contextual data into a compelling narrative."
        ),
        "expected_output": (
            "A comprehensive speech that seamlessly combines the company and country information into a clear, engaging narrative suitable for an invited guest."
        ),
    },
    "quality_assurance": {
        "description": (
            "Review and refine the generated speech to ensure it meets the highest standards of clarity, engagement, and factual accuracy. "
            "Ensure the speech is well-formatted in markdown with appropriate headers and bullet points."
        ),
        "expected_output": (
            "A finalized version of the speech, thoroughly reviewed and formatted in markdown, that is clear, engaging, and informative."
        ),
    },
}

# ================================
# Pydantic Model for Output
# ================================
class SpeechOutput(BaseModel):
    speech: str = Field(..., description="The final generated speech in markdown format.")

# ================================
# Utility function to print the speech neatly
# ================================
def print_speech(speech_output: SpeechOutput) -> None:
    # Use model_dump() to avoid the deprecated dict() method.
    speech = speech_output.model_dump().get("speech", "")
    if not speech:
        logging.warning("No speech found in the output.")
        return
    wrapped_speech = textwrap.fill(speech, width=80)
    print("\nFinal Speech:")
    print("=" * 80)
    print(wrapped_speech)
    print("=" * 80)

# ================================
# Main Function
# ================================
def main():
  
    load_env()

    # Set environment variable for the LLM model 
    #os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'
    groq_llm = "groq/llama-3.3-70b-versatile"

    # ---------------------------
    # Create Agents with Tools
    # ---------------------------
    company_info_agent = Agent(
        config=agents_config['company_info_agent'],
        tools=[SerperDevTool(), ScrapeWebsiteTool(), WebsiteSearchTool()],
        llm=groq_llm,
    )

    country_info_agent = Agent(
        config=agents_config['country_info_agent'],
        tools=[SerperDevTool(), WebsiteSearchTool()],
        llm=groq_llm,
    )

    speech_generator_agent = Agent(
        config=agents_config['speech_generator_agent'],
        tools=[SerperDevTool(), WebsiteSearchTool()],
        llm=groq_llm,
    )

    quality_assurance_agent = Agent(
        config=agents_config['quality_assurance_agent'],
    )

    # ---------------------------
    # Create Tasks
    # ---------------------------
    retrieve_company_info_task = Task(
        config=tasks_config['retrieve_company_info'],
        agent=company_info_agent
    )

    retrieve_country_info_task = Task(
        config=tasks_config['retrieve_country_info'],
        agent=country_info_agent
    )

    generate_speech_task = Task(
        config=tasks_config['generate_speech'],
        agent=speech_generator_agent,
        context=[retrieve_company_info_task, retrieve_country_info_task]
    )

    quality_assurance_task = Task(
        config=tasks_config['quality_assurance'],
        agent=quality_assurance_agent,
        output_pydantic=SpeechOutput
    )

    # ---------------------------
    # Assemble Crew and Execute Tasks
    # ---------------------------
    speech_creation_crew = Crew(
        agents=[
            company_info_agent,
            country_info_agent,
            speech_generator_agent,
            quality_assurance_agent,
        ],
        tasks=[
            retrieve_company_info_task,
            retrieve_country_info_task,
            generate_speech_task,
            quality_assurance_task
        ],
        verbose=True
    )

    # Define input details for the speech 
    input_subject = {
        'company': 'Ethiopian Artificial Intelligence Institute',
        'country': 'Ethiopia'
    }
    logging.info("Kicking off the speech creation crew.")
    result = speech_creation_crew.kickoff(inputs=input_subject)
    logging.info("Crew execution completed.")

    # Debug: Print raw result 
    print(result)

    # ---------------------------
    # Display the Final Speech
    # ---------------------------
    if hasattr(result, 'pydantic'):
        speech_output: SpeechOutput = result.pydantic
        
        display(Markdown(speech_output.speech))
        
        print_speech(speech_output)
    else:
        logging.error("The result does not contain a valid 'pydantic' attribute.")

# Execute the main function to run the workflow
main()


2025-03-05 15:29:13,183 - 140230655523712 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Company Information Specialist
## Task: Retrieve comprehensive company information including background, industry details, achievements, and overall impact for Ethiopian Artificial Intelligence Institute.


# Agent: Company Information Specialist
## Thought: Thought: To gather comprehensive information about the Ethiopian Artificial Intelligence Institute, I need to start by searching the internet for available data on the institute. This will give me a broad overview and possibly direct me to specific websites or sources where I can find more detailed information.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"Ethiopian Artificial Intelligence Institute\"}"
## Tool Output: 

Search results: Title: Artificial Intelligence Institute: Home
Link: https://aii.et/
Snippet: artificial intelligence institute In 2030 to be a state-of-the-art National AI Research and Development Center with Excellence and Key Role in Creating ...
---
Title: Ethiopian Artificia

 

Action 'None' don't exist, these are the only available Actions:
Tool Name: Search the internet(**kwargs: Any) -> Any
Tool Description: Search the internet(search_query: 'string') - A tool that can be used to search the internet with a search_query. search_query: 'Mandatory search query you want to use to search the internet'
Tool Arguments: {'name': FieldInfo(annotation=str, required=True), 'description': FieldInfo(annotation=str, required=False, default=''), 'args_schema': FieldInfo(annotation=type[BaseModel], required=True, description='The tool schema.', metadata=[SkipValidation()]), 'return_direct': FieldInfo(annotation=bool, required=False, default=False), 'verbose': FieldInfo(annotation=bool, required=False, default=False), 'callbacks': FieldInfo(annotation=Union[list[BaseCallbackHandler], BaseCallbackManager, NoneType], required=False, default=None, exclude=True), 'callback_manager': FieldInfo(annotation=Union[BaseCallbackManager, NoneType], required=False, default=None, des



# Agent: Company Information Specialist
## Final Answer: 
The Ethiopian Artificial Intelligence Institute was established on July 29, 2019, as a center that aims to catalyze the development of artificial intelligence in Ethiopia while encouraging its application to create social and economic value. The center's vision is to be a world-class institute with impactful national and regional influence, forefront in advancing knowledge in Artificial Intelligence, and leveraging AI-enabled solutions at the national and international levels.

The institute's mission is to catalyze the development and utilization of artificial intelligence to solve societal and economic problems and provide effective AI-enabled solutions. The objectives of the institute, as stated in Proclamation No. 1205/2022, Article 3, Part 2, Sub-article 3.1, include developing the necessary framework and standard for the development, utilization, and governance of artificial intelligence, developing and utilizing artific



# Agent: Country Information Specialist
## Thought: Thought: To gather key data on the economy, culture, and business environment of Ethiopia, I should start by searching for general information about the country's economy, cultural trends, and business landscape. This will provide a foundation for further research and help identify areas that require more in-depth analysis.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"Ethiopia economy culture business environment\"}"
## Tool Output: 

Search results: Title: Ethiopia Overview: Development news, research, data | World Bank
Link: https://www.worldbank.org/en/country/ethiopia/overview
Snippet: A robust private sector is pivotal to Ethiopia's economic future, with the International Finance Corporation (IFC) and Multilateral Investment ...
---
Title: Ethiopia - Market Overview - International Trade Administration
Link: https://www.trade.gov/country-commercial-guides/ethiopia-market-overview
Snippet: Discusses k

 

I encountered an error while trying to use the tool. This was the error: 1 validation error for SerperDevToolSchema
search_query
  Field required [type=missing, input_value={'name': 'speech structur...ctive speech structure'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing.
 Tool Search the internet accepts these inputs: Search the internet(search_query: 'string') - A tool that can be used to search the internet with a search_query. search_query: 'Mandatory search query you want to use to search the internet'



# Agent: Speech Generator
## Using tool: Search the internet
## Tool Input: 
"{\"name\": \"speech structure\", \"description\": \"effective speech structure\"}"
## Tool Output: 

I encountered an error while trying to use the tool. This was the error: 1 validation error for SerperDevToolSchema
search_query
  Field required [type=missing, input_value={'name': 'speech structur...ctive speech structure'}, input_type=dict]
    For f



# Agent: Speech Generator
## Thought: Thought: With the information gathered on speech structure and elements of an engaging speech, I can now create a well-structured and engaging speech for an invited guest. The speech will include an introduction, body, and conclusion, and will incorporate elements such as clear and concise main points, verbal and nonverbal cues, and personal anecdotes.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"tips for public speaking\"}"
## Tool Output: 

Search results: Title: 10 Tips for Improving Your Public Speaking Skills
Link: https://professional.dce.harvard.edu/blog/10-tips-for-improving-your-public-speaking-skills/
Snippet: Here Are My 10 Tips for Public Speaking: · 1. Nervousness Is Normal. · 2. Know Your Audience. · 3. Organize Your Material in the Most Effective ...
---
Title: 11 Public Speaking Tips: How to Relax, Focus, and Shine at Your ...
Link: https://www.purdueglobal.edu/blog/student-life/public-speaking-tips/
Sn



# Agent: Speech Generator
## Final Answer: 
Distinguished guests, ladies, and gentlemen, I am honored to be speaking with you today about the Ethiopian Artificial Intelligence Institute and its role in promoting economic growth and development in Ethiopia.

As we gather here today, I want to take a moment to reflect on the significance of this institution and its potential to drive positive change in our country. The Ethiopian Artificial Intelligence Institute was established in 2019 with the objective of catalyzing the development and application of artificial intelligence in Ethiopia. The institute's vision is to be a world-class institute with impactful national and regional influence, forefront in advancing knowledge in Artificial Intelligence, and leveraging AI-enabled solutions at the national and international levels.

The institute's mission is to catalyze the development and utilization of artificial intelligence to solve societal and economic problems and provide effective 

# Speech on the Ethiopian Artificial Intelligence Institute

**Distinguished guests, ladies, and gentlemen,**

I am honored to be speaking with you today about the **Ethiopian Artificial Intelligence Institute** and its pivotal role in promoting economic growth and development in Ethiopia.

## Significance of the Ethiopian Artificial Intelligence Institute

As we gather here today, let us take a moment to reflect on the significance of this institution and its potential to drive positive change in our country. Established in **2019**, the Ethiopian Artificial Intelligence Institute aims to catalyze the development and application of artificial intelligence in Ethiopia. 

### Vision and Mission

The institute's vision is to become a world-class institution with impactful national and regional influence. It seeks to be at the forefront of advancing knowledge in artificial intelligence (AI) while leveraging AI-enabled solutions both nationally and internationally.

The mission of the Ethiopian Artificial Intelligence Institute focuses on:
- Catalyzing the development and utilization of artificial intelligence.
- Solving societal and economic problems through effective AI-enabled solutions.

With a strong dedication to innovating AI-driven solutions, the institute has forged partnerships with various stakeholders, including:
- Government institutions
- Private sector companies
- International organizations

## Impactful Areas of Work

One of the key areas where the Ethiopian Artificial Intelligence Institute is making a significant impact is in the development of AI-enabled solutions for **disease prediction, detection, and management**. The institute has successfully developed a disease prediction and detection system that has the potential to revolutionize healthcare in Ethiopia.

In addition, the institute has launched various initiatives such as:
- **EFPApp**: A mobile application developed for the Ethiopian Federal Police, allowing citizens to report crimes easily.
- Projects focused on customer chatbots and revenue assurance in tax collections.

### Global Partnerships

The institute has also established partnerships with renowned international organizations, including the **Massachusetts Institute of Technology (MIT)**, to advance talent development and drive research and development in artificial intelligence within Ethiopia.

## Looking to the Future

As we look to the future, it is evident that the Ethiopian Artificial Intelligence Institute has a critical role to play in driving economic growth and development. Its work holds the promise of making a significant impact on Ethiopia's economic and social landscape. Furthermore, its partnerships with international organizations will facilitate the exchange of innovative ideas, technologies, and expertise needed to accelerate the development of artificial intelligence in our country.

## Conclusion

In conclusion, the Ethiopian Artificial Intelligence Institute stands as a pioneering center for research and development in artificial intelligence in Ethiopia. The institute's initiatives have the potential to transform the economic and social landscape of our nation. 

I am confident that, with continued support from our partners and stakeholders, the Ethiopian Artificial Intelligence Institute will not only thrive but will also make a profound and positive difference in the lives of all Ethiopians.

**Thank you.**


Final Speech:
# Speech on the Ethiopian Artificial Intelligence Institute  **Distinguished
guests, ladies, and gentlemen,**  I am honored to be speaking with you today
about the **Ethiopian Artificial Intelligence Institute** and its pivotal role
in promoting economic growth and development in Ethiopia.  ## Significance of
the Ethiopian Artificial Intelligence Institute  As we gather here today, let us
take a moment to reflect on the significance of this institution and its
potential to drive positive change in our country. Established in **2019**, the
Ethiopian Artificial Intelligence Institute aims to catalyze the development and
application of artificial intelligence in Ethiopia.   ### Vision and Mission
The institute's vision is to become a world-class institution with impactful
national and regional influence. It seeks to be at the forefront of advancing
knowledge in artificial intelligence (AI) while leveraging AI-enabled solutions
both nationally and internationally.  The missi

In [7]:
import os
import logging
import textwrap
from typing import List
from pydantic import BaseModel, Field


from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool, WebsiteSearchTool, ScrapeWebsiteTool


from IPython.display import display, Markdown


def load_env():
    
    pass

# Set up logging configuration
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# ================================
# Agent Configurations
# ================================
agents_config = {
    "company_info_agent": {
        "role": "Company Information Specialist",
        "goal": "Extract comprehensive background, industry details, key achievements, and overall impact for a given company.",
        "backstory": (
            "As a specialist in company analysis, you gather and summarize detailed information about companies to provide "
            "insights that will help craft an engaging speech."
        ),
        "allow_delegation": False,
        "verbose": True,
    },
    "country_info_agent": {
        "role": "Country Information Specialist",
        "goal": "Gather key data on the economy, culture, and business environment of a given country.",
        "backstory": (
            "As an expert in country profiling, you analyze essential national metrics and cultural aspects to provide valuable "
            "context for a well-rounded speech."
        ),
        "allow_delegation": False,
        "verbose": True,
    },
    "speech_generator_agent": {
        "role": "Speech Generator",
        "goal": (
            "Create a well-structured, engaging, and informative speech for an invited guest by integrating company and country data."
        ),
        "backstory": (
            "As an experienced speechwriter, you craft compelling speeches that seamlessly combine detailed company information "
            "with insights about the country to produce a coherent and engaging narrative."
        ),
        "allow_delegation": False,
        "verbose": True,
    },
    "quality_assurance_agent": {
        "role": "Content Quality Assurer",
        "goal": (
            "Review and refine the generated speech to ensure clarity, factual accuracy, and overall engagement, while ensuring "
            "the content is well-formatted in markdown."
        ),
        "backstory": (
            "As the quality assurance specialist, you meticulously proofread and optimize content to meet high standards of quality and "
            "presentation."
        ),
        "allow_delegation": False,
        "verbose": True,
    },
}

# ================================
# Task Configurations
# ================================
tasks_config = {
    "retrieve_company_info": {
        "description": (
            "Retrieve comprehensive company information including background, industry details, achievements, and overall impact for {company}."
        ),
        "expected_output": (
            "A detailed summary of {company} covering its background, industry position, notable achievements, and overall impact."
        ),
    },
    "retrieve_country_info": {
        "description": (
            "Gather key data on the economy, culture, and business environment for {country}."
        ),
        "expected_output": (
            "A detailed summary of {country} covering economic trends, cultural highlights, and the business environment."
        ),
    },
    "generate_speech": {
        "description": (
            "Based on the company and country information retrieved, generate a well-structured and engaging speech for an invited guest. "
            "Integrate insights and contextual data into a compelling narrative."
        ),
        "expected_output": (
            "A comprehensive speech that seamlessly combines the company and country information into a clear, engaging narrative suitable for an invited guest."
        ),
    },
    "quality_assurance": {
        "description": (
            "Review and refine the generated speech to ensure it meets the highest standards of clarity, engagement, and factual accuracy. "
            "Ensure the speech is well-formatted in markdown with appropriate headers and bullet points."
        ),
        "expected_output": (
            "A finalized version of the speech, thoroughly reviewed and formatted in markdown, that is clear, engaging, and informative."
        ),
    },
}

# ================================
# Pydantic Model for Output
# ================================
class SpeechOutput(BaseModel):
    speech: str = Field(..., description="The final generated speech in markdown format.")

# ================================
# Utility function to print the speech neatly
# ================================
def print_speech(speech_output: SpeechOutput) -> None:
    
    speech = speech_output.model_dump().get("speech", "")
    if not speech:
        logging.warning("No speech found in the output.")
        return
    wrapped_speech = textwrap.fill(speech, width=80)
    print("\nFinal Speech:")
    print("=" * 80)
    print(wrapped_speech)
    print("=" * 80)

# ================================
# Main Function
# ================================
def main():
    # Load environment variables
    load_env()

    # Set environment variable for the LLM model (customize as needed)
    #os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'
    groq_llm = "groq/llama-3.3-70b-versatile"

    # ---------------------------
    # Create Agents with Tools
    # ---------------------------
    company_info_agent = Agent(
        config=agents_config['company_info_agent'],
        tools=[SerperDevTool(), ScrapeWebsiteTool(), WebsiteSearchTool()],
        llm=groq_llm,
    )

    country_info_agent = Agent(
        config=agents_config['country_info_agent'],
        tools=[SerperDevTool(), WebsiteSearchTool()],
        llm=groq_llm,
    )

    speech_generator_agent = Agent(
        config=agents_config['speech_generator_agent'],
        tools=[SerperDevTool(), WebsiteSearchTool()],
        llm=groq_llm,
    )

    quality_assurance_agent = Agent(
        config=agents_config['quality_assurance_agent'],
    )

    # ---------------------------
    # Create Tasks
    # ---------------------------
    retrieve_company_info_task = Task(
        config=tasks_config['retrieve_company_info'],
        agent=company_info_agent
    )

    retrieve_country_info_task = Task(
        config=tasks_config['retrieve_country_info'],
        agent=country_info_agent
    )

    generate_speech_task = Task(
        config=tasks_config['generate_speech'],
        agent=speech_generator_agent,
        context=[retrieve_company_info_task, retrieve_country_info_task]
    )

    quality_assurance_task = Task(
        config=tasks_config['quality_assurance'],
        agent=quality_assurance_agent,
        output_pydantic=SpeechOutput
    )

    # ---------------------------
    # Assemble Crew and Execute Tasks
    # ---------------------------
    speech_creation_crew = Crew(
        agents=[
            company_info_agent,
            country_info_agent,
            speech_generator_agent,
            quality_assurance_agent,
        ],
        tasks=[
            retrieve_company_info_task,
            retrieve_country_info_task,
            generate_speech_task,
            quality_assurance_task
        ],
        verbose=True
    )

    # Define input details for the speech 
    input_subject = {
        'company': 'Tesla, Inc.',
        'country': 'United States'
    }
    logging.info("Kicking off the speech creation crew.")
    result = speech_creation_crew.kickoff(inputs=input_subject)
    logging.info("Crew execution completed.")

    # Debug: Print raw result 
    print(result)

    # ---------------------------
    # Display the Final Speech
    # ---------------------------
    if hasattr(result, 'pydantic'):
        speech_output: SpeechOutput = result.pydantic
        
        display(Markdown(speech_output.speech))
        
        print_speech(speech_output)
    else:
        logging.error("The result does not contain a valid 'pydantic' attribute.")

# Execute the main function to run the workflow
main()

2025-03-05 15:32:08,508 - 140230655523712 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Company Information Specialist
## Task: Retrieve comprehensive company information including background, industry details, achievements, and overall impact for Tesla, Inc..


# Agent: Company Information Specialist
## Thought: Thought: To gather comprehensive information about Tesla, Inc., I should start by searching the internet for general information about the company, including its background, industry, achievements, and overall impact.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"Tesla, Inc. company information\"}"
## Tool Output: 

Search results: Title: About - Tesla
Link: https://www.tesla.com/about
Snippet: Headquartered in Texas, we operate six huge, vertically integrated factories across three continents. With over 100,000 employees, our teams design, build, sell ...
---
Title: Tesla, Inc. - Wikipedia
Link: https://en.wikipedia.org/wiki/Tesla,_Inc.
Snippet: Tesla, Inc is an American multinational automotive and clean energy company. Headq



# Agent: Country Information Specialist
## Thought: Thought: To gather key data on the economy, culture, and business environment of the United States, I need to search for relevant information on the internet. I will start by searching for an overview of the United States economy.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"United States economy overview\"}"
## Tool Output: 

Search results: Title: U.S. Economy at a Glance | U.S. Bureau of Economic Analysis (BEA)
Link: https://www.bea.gov/news/glance
Snippet: Real gross domestic product (GDP) increased at an annual rate of 2.3 percent in the fourth quarter of 2024, according to the second estimate. In the third ...
---
Title: United States Economy Overview: Outlook & Reports
Link: https://www.focus-economics.com/countries/united-states/
Snippet: The United States' Macroeconomic Analysis: Nominal GDP of USD 29,179 billion in 2024. GDP per capita of USD 86,635 compared to the global average of USD 10,589.




# Agent: Country Information Specialist
## Final Answer: 
The United States is a federal republic consisting of 50 states, a federal district, and several territories. It is a country with a diverse geography, climate, and culture, and is home to a wide range of ethnicities, languages, and belief systems. The country has a highly developed economy and is one of the world's leading economic powers.

The US economy is driven by consumer spending, business investment, and government spending, and is influenced by a range of factors, including monetary policy, fiscal policy, and global events. The country has a highly developed financial system, with a large and sophisticated banking sector, a well-developed stock market, and a significant bond market. The US dollar is one of the most widely traded and widely held currencies in the world, and is seen as a safe-haven asset during times of economic uncertainty.

American culture is a diverse and complex phenomenon, shaped by the country's 

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  4.06it/s]




# Agent: Speech Generator
## Thought: Based on the tips from the search results, I can now create a speech that includes the information about Tesla, Inc. and the United States.
Thought: With these tips in mind, I can create a speech that provides an overview of Tesla, Inc., a pioneering company in the automotive and clean energy industries, and the United States, a country with a diverse geography, climate, and culture.
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"entrepreneurship and innovation in the US\", \"website\": \"https://www.forbes.com/\"}"
## Tool Output: 
Relevant Content:
EditionU.S.AsiaEuropeThis AI Cloud Provider Just Minted Four New BillionairesByPhoebe Liu, Forbes StaffFrom stockpiling Nvidia chips to selling computing power: how these former commodity traders have already made a bundle from CoreWeave, which just filed to go public. This Founder’s Gene Therapy Company Helped Blind Kids See AgainZandy Forbes’ biotech company Meira



# Agent: Content Quality Assurer
## Final Answer: 
```markdown
# Transforming the Future: The Story of Tesla, Inc. and the United States

Ladies and gentlemen, esteemed guests, and fellow innovators, 

I am honored to stand before you today to discuss a company that has truly revolutionized the automotive and clean energy industries, as well as the country that has paved the way for entrepreneurship and innovation—**the United States of America**.

## The Story of Tesla, Inc.

Today, I want to share with you the remarkable story of **Tesla, Inc.**, a pioneering company that has made significant contributions to the development of sustainable energy solutions and the automotive industry. Founded in **2003** by American entrepreneurs **Elon Musk, Martin Eberhard, Marc Tarpenning, JB Straubel, and Ian Wright**, Tesla has emerged as a driving force in the electric vehicle (EV) market, holding a market share of over **24%** globally.

### Tesla's Impact on the Automotive Industry

- Revol

# Transforming the Future: The Story of Tesla, Inc. and the United States

Ladies and gentlemen, esteemed guests, and fellow innovators, 

I am honored to stand before you today to discuss a company that has truly revolutionized the automotive and clean energy industries, as well as the country that has paved the way for entrepreneurship and innovation—**the United States of America**.

## The Story of Tesla, Inc.

Today, I want to share with you the remarkable story of **Tesla, Inc.**, a pioneering company that has made significant contributions to the development of sustainable energy solutions and the automotive industry. Founded in **2003** by American entrepreneurs **Elon Musk, Martin Eberhard, Marc Tarpenning, JB Straubel, and Ian Wright**, Tesla has emerged as a driving force in the electric vehicle (EV) market, holding a market share of over **24%** globally.

### Tesla's Impact on the Automotive Industry

- Revolutionized the perception and viability of electric vehicles
- Introduced cutting-edge technologies and innovations in EV design 
- Paved the path for sustainable transportation solutions

## The United States: A Catalyst for Innovation

What makes Tesla's story even more remarkable is the country in which it was founded—the **United States**. 

### The Landscape of the US

- **Diverse Geography, Climate, and Culture**: A melting pot of ethnicities, languages, and beliefs
- **Economic Powerhouse**: One of the world's leading economies 

### Key Economic Drivers

The US economy thrives on:
- **Consumer Spending**
- **Business Investment**
- **Government Spending**

These factors are influenced by:
- **Monetary Policy**
- **Fiscal Policy**
- **Global Events**

## Current Economic Outlook

As we look at the current state of the US economy, it's poised for growth:
- A **strong labor market**
- **Low unemployment rates**
- **Robust consumer confidence**

The **Federal Reserve** has successfully softened the landing and mitigated the economic contraction typically experienced during monetary tightening cycles. The job market is expected to remain robust, with further modest gains anticipated, provided monetary policy stays appropriately calibrated and that no significant economic risks arise.

## Tesla's Mission in Today's Context

In this promising environment, Tesla's mission to **accelerate the world’s transition to sustainable energy** is more relevant than ever. Their commitment involves producing not only electric vehicles but also:
- **Solar power systems**
- **Energy storage products**

### Transforming Energy and Transportation

Tesla's innovative products and technologies have:
- Transformed public perception about energy 
- Inspired a new generation of entrepreneurs and innovators focused on sustainability

## Looking Forward

As we gaze into the future, it is evident that the United States will continue to play a significant role in shaping the global economy and driving innovation. 

### The Path Ahead

The country's strong emphasis on:
- **Entrepreneurship**
- **Innovation**
- **Competition**

will persistently fuel growth and job creation. Furthermore, its well-structured financial system will provide vital funding for startups and small businesses to thrive.

## Conclusion

In conclusion, the story of **Tesla, Inc.** and the **United States** serves as a testament to the power of innovation and entrepreneurship. It exemplifies how determination, hard work, and a steadfast commitment to sustainability can create a brighter future for ourselves and generations to come.

Thank you for your attention.


Final Speech:
# Transforming the Future: The Story of Tesla, Inc. and the United States
Ladies and gentlemen, esteemed guests, and fellow innovators,   I am honored to
stand before you today to discuss a company that has truly revolutionized the
automotive and clean energy industries, as well as the country that has paved
the way for entrepreneurship and innovation—**the United States of America**.
## The Story of Tesla, Inc.  Today, I want to share with you the remarkable
story of **Tesla, Inc.**, a pioneering company that has made significant
contributions to the development of sustainable energy solutions and the
automotive industry. Founded in **2003** by American entrepreneurs **Elon Musk,
Martin Eberhard, Marc Tarpenning, JB Straubel, and Ian Wright**, Tesla has
emerged as a driving force in the electric vehicle (EV) market, holding a market
share of over **24%** globally.  ### Tesla's Impact on the Automotive Industry
- Revolutionized the perception and viability of electric ve